In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from datetime import datetime
import tensorflow as tf
random.seed(2025)
np.random.seed(2025)
tf.random.set_seed(2025)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Dropout, Input
from keras.optimizers import Adam
import os
from sklearn.model_selection import TimeSeriesSplit
tf.config.experimental.enable_op_determinism()

In [ ]:
data = pd.read_pickle('../input/data_clean.pkl')


In [ ]:
data = data.rename(columns={'Area [m2]': 'Area'})
data

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
building_names = data['Building_ID'].unique().tolist()
building_names

In [ ]:
# Train the DDQN agent
batch_size = 64
epochs = 100
window_size = 24
episodes = 5

In [ ]:
# Defining the Double Deep Q-Network agent
class DDQNAgent:
    def __init__(self, state_size, action_size, n_features):
        self.state_size = state_size
        self.n_features = n_features
        self.action_size = action_size
        self.memory = []  # For experience replay
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add((LSTM(50, activation='relu', input_shape=(self.state_size, self.n_features), return_sequences=True)))
        model.add(Dropout(0.2))
        model.add(LSTM(50, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(optimizer='adam', loss='mse')
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):

        act_values = self.model.predict(state, verbose=0)

        return act_values[0]

    def replay(self, batch_size):

        indices = np.random.choice(len(self.memory), batch_size, replace=False)

        minibatch = [self.memory[idx] for idx in indices]

        for state, action, reward, next_state, done in minibatch:

            target = reward

            print('target: ', target)

            target_f = self.model.predict(state)

            print('target_f: ', target_f)

            target_f[0] = target

            print('target_f[0]: ', target_f[0])

            print('target_f: ', target_f)

            self.model.fit(state, [target_f], epochs=1, verbose=1)

In [ ]:
cwd = os.getcwd()
print(cwd)

from datetime import datetime

now = datetime.now()
dtime = now.strftime('%H-%M-%S-%d-%m-%Y')
print('date and time: ', dtime)

report_dir = cwd + "/results" + "/lstm_ddqn/" + str("wz") + str(window_size) + str("_e") + str(epochs) + str(
    "_epi_") + str(episodes) + str("_batchsize_") + str(batch_size) + "_" + dtime + ".csv"

print('report_dir: ', report_dir)

In [ ]:
def safe_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

In [ ]:
with open(report_dir, "a") as text_file:
    for building_idx in range(len(building_names)):

        building = building_names[building_idx]
        print(f"Process building {building_idx+1}/{len(building_names)}: {building}")
        
        idx_start = building_idx * 8760
        idx_end = (building_idx + 1) * 8760
        building_data = data.iloc[idx_start:idx_end, :].copy()

        building_data = building_data.drop(columns=['Building_ID'])
        feature_columns = building_data.drop(columns=['Timestamp']).columns
        scaler = MinMaxScaler().fit(building_data.drop(columns=['Timestamp']))
        start = datetime(2023, 1, 1)
        end = datetime(2023, 3, 1)
        
        train_df = building_data[(building_data['Timestamp'] >= start) & (building_data['Timestamp'] < end)]
        train_df = train_df.drop(columns=['Timestamp'])
        train_scaled = scaler.transform(train_df)

        
        test_df = building_data[(building_data['Timestamp'] >= end)]
        test_df = test_df.drop(columns=['Timestamp'])
        test_scaled = scaler.transform(test_df)

        window_size = 24
        X_train, y_train = [], []
        for i in range(window_size, len(train_scaled)):
            X_train.append(train_scaled[i-window_size:i])
            y_train.append(train_scaled[i][0])  # Assuming first column is the target (energy consumption)
        X_train, y_train = np.array(X_train), np.array(y_train)

        X_validation, y_validation = [], []
        for i in range(window_size, len(test_scaled)):
            X_validation.append(test_scaled[i-window_size:i])
            y_validation.append(test_scaled[i][0])
        X_validation, y_validation = np.array(X_validation), np.array(y_validation)

        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

        # Initialize DDQN Agent
        state_size = window_size
        feature_size = X_train.shape[2]
        action_size = 1
        agent = DDQNAgent(state_size, action_size, feature_size)
        
        # Callbacks
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
        callbacks = [early_stopping, reduce_lr]
        
        # Train the DDQN agent
        for episode in range(episodes):
            print(f"Episode {episode+1}/{episodes}")

            tscv = TimeSeriesSplit(n_splits=4)

            for train_index, val_index in tscv.split(X_train):
                pass

            agent.model.fit(
                X_train[train_index], y_train[train_index],
                epochs=epochs,
                batch_size=32,
                validation_data=(X_train[val_index], y_train[val_index]),
                callbacks=callbacks,
            )

            for i in range(len(X_train) - 1):
                state = X_train[i].reshape(1, window_size, feature_size)
                action = agent.act(state)
                next_state = X_train[i + 1].reshape(1, window_size, feature_size)
                if (np.abs(y_train[i] - y_train[i + 1]) > 0.001):
                    reward = y_train[i]
                    done = False
                else:
                    reward = y_train[i]
                    done = True
                agent.remember(state, action, reward, next_state, done)

            # Test set predictions
            current_window = X_train[-1].copy()
            y_test_pred = []
            for i in range(len(test_scaled)):
                input_window = current_window.reshape(1, window_size, feature_size)
                input_window = np.nan_to_num(input_window, nan=0)
                y_pred = agent.model.predict(input_window, verbose=0)[0][0]
                y_test_pred.append(y_pred)
                next_step = test_scaled[i].copy()
                next_step[0] = y_pred
                current_window = np.vstack([current_window[1:], next_step])

            if len(y_test_pred) == len(test_scaled):
                test_true = test_scaled[:, 0]
                dummy_pred = np.zeros((len(y_test_pred), len(scaler.scale_)))
                dummy_true = np.zeros((len(test_true), len(scaler.scale_)))
                dummy_pred[:, 0] = np.array(y_test_pred)
                dummy_true[:, 0] = test_true
                dummy_pred = np.clip(dummy_pred, 0, 1)
                dummy_pred = np.nan_to_num(dummy_pred)
                dummy_true = np.clip(dummy_true, 0, 1)
                dummy_true = np.nan_to_num(dummy_true)
                
                y_test_pred_inv = scaler.inverse_transform(dummy_pred)[:, 0]
                y_test_inv = scaler.inverse_transform(dummy_true)[:, 0]
                
                mape = safe_mape(y_test_pred_inv, y_test_inv)
                text_file.write(f"Building {building} - Test MAPE: {mape:.2f}%\n")
                print(f"Test MAPE: {mape:.2f}%")
                
                # Plot test results
                plt.figure(figsize=(14, 6))
                plt.plot(y_test_inv, label='True Usage', color='blue')
                plt.plot(y_test_pred_inv, label='Predicted Usage', color='orange')
                plt.title(f"Building {building} - Recursive Forecast on Test Set")
                plt.xlabel("Hour")
                plt.ylabel("Electricity Consumption (kWh)")
                plt.legend()
                plt.grid(True)
                plt.tight_layout()
                plt.show()
            else:
                print("Size doesn't match!")

            if len(agent.memory) > batch_size:
                agent.replay(batch_size)